# Interactive Maps for Google Earth Engine Python API

### This notebook is a showcase of a new feature of `geetools`, and it is still under active development on https://github.com/gee-community/gee_tools. There are 2 options:

## 1. maptool
### Based on https://github.com/mccarthyryanc/folium_gee but improved in order to 'emulate' the behavior of Map in the Code Editor. 
#### Pros:
1. This module can be use to generate html for other purposes like webapps, etc
2. Can change zoom with mouse scroll

#### Cons:
1. When using this module you have to complete the Map (addLayer, etc), and at the end use `Map.show()` to show the map. Can't add more layers afterwards.
2. Takes a long time to show the map

## 2. ipymap
### Based on https://github.com/gee-community/ee-jupyter-contrib/blob/master/examples/getting-started/display-interactive-map.ipynb
#### Pros and cons are the opposite to `maptool`

#### *(I'll try to keep this notebook up to date)*

# Make imports

In [1]:
import ee
ee.Initialize()

# Get an image

In [2]:
col = ee.ImageCollection('COPERNICUS/S2')
site = ee.Geometry.Point([-72, -42])
col = col.filterBounds(site).filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 40)
i = ee.Image(col.first())
igeom = i.geometry()

# define visualization parameters (dict)

In [3]:
visParam = {'bands':['B8', 'B11', 'B4'], 'min':0, 'max':5000}

## define inspection
### `addLayer` has a parameter called `inspect` that defines de content for a pop up, which is a dict with the following keys:
1. data: eeObject from where to get the data
2. reducer: can be 'first', 'mean', 'media' and 'sum'
3. scale: scale to use in the reduction. If not provided, it uses the nominalScale for the image's first band

In [4]:
inspect = {'data':i, 'reducer':'mean'}

# Plot the image into an interactive Map using `maptool` (Folium)

In [5]:
from geetools.ui import maptool
Map = maptool.Map()

Map.addLayer(i, visParam, 'Sentinel 2 Patagonia')
Map.centerObject(i)

Map.addLayer(igeom, name='Image boundries', inspect=inspect)

Map.show()

# Plot the image into an interactive Map using `ipymap` (ipyleaflet)
## This Map has extra features:
1. Inspector
2. Tasks
3. Assets

### At the moment, the only one working is the inspector

## First plot an empty Map (you can add stuff here too)

In [6]:
from geetools import ui
Map2 = ui.Map()

# you can do also:
# from geetools.ui import ipymap
# Map2 = ipymap.Map()

In [7]:
Map2.show()

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

## addLayer

In [8]:
Map2.addLayer(i, visParam, 'Sentinel 2 Patagonia')

In [9]:
Map2.addLayer(i, {'bands':['B8'], 'min':0, 'max':5000}, 'just B8')

## Center an Image

In [10]:
Map2.centerObject(i)

## addLayer Geometry

In [11]:
Map2.addLayer(igeom, name='Image boundries')

## Center a Geometry

In [12]:
Map2.centerObject(igeom)

## Get Map Center

In [13]:
center = Map2.getCenter()
center.getInfo()

{'coordinates': [-71.91248573714033, -42.15901782145145],
 'geodesic': True,
 'type': 'Point'}

## Get Map Bounds

In [14]:
bounds = Map2.getBounds()
bounds.getInfo()

EEException: The Geometry.Rectangle constructor requires 2 points or 4 coordinates.

## Remove a layer

In [15]:
Map2.removeLayer('Sentinel 2 Patagonia')

# TABS
## You can add a custom Tab with a custom handler. The handler is a function with 4 main parameters:
- **type:** the interaction type. Can be 'click', 'mouseover', etc
- **coordinates:** the coordinates where the interaction has taken place. If you have used ipyleaflet before, take in consideraton that coordinates are inverted (to match GEE format): [longitud, latitude]
- **widget:** The widget inside the Tab. Defaults to an empty HTML widget
- **map:** the Map instance. You can apply any of its methods, or get any of its properties

In [16]:
print(Map2.addTab.__doc__)

 Add a Tab to the Panel. The handler is for the Map

        :param name: name for the new tab
        :type name: str
        :param handler: handle function for the new tab. Arguments of the
        function are:

          - type: the type of the event (click, mouseover, etc..)
          - coordinates: coordinates where the event occurred [lon, lat]
          - widget: the widget inside the Tab
          - map: the Map instance

        :param widget: widget inside the Tab. Defaults to HTML('')
        :type widget: ipywidgets.Widget
        


In [17]:
def test_handler(**change):    
    # PARAMS
    ty = change['type']
    coords = change['coordinates']
    wid = change['widget']
    themap = change['map']
        
    if ty == 'click':  # If interaction was a click
        # Loading message before sending a request to EE
        wid.value = 'Loading...'
        # Map's bounds
        bounds = themap.getBounds().getInfo()['coordinates']
        # Change Widget Value
        wid.value = "You have clicked on {} and map's bounds are {}".format(coords, bounds)

Map2.addTab('TestTAB', test_handler)
print("Check out the Map!")

Check out the Map!
